<a href="https://colab.research.google.com/github/s-ryuri/TIL/blob/main/%EA%B3%B5%EB%AA%A8%EC%A0%84EDA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Colab/9월16일

import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import warnings
#ignore warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn')

plt.rc('font', family='NanumBarunGothic') 
def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'
    
    font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    os.system(font_download_cmd)
    os.system(unzip_cmd)
    
    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    font_name = fm.FontProperties(fname=font_files[0]).get_name()
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

font_download_url = "https://fonts.google.com/download?family=Noto%20Sans%20KR"
change_matplotlib_font(font_download_url)

In [ ]:
powerfactor = pd.read_csv('new역률평균')
electric = pd.read_csv('new전류평균')
voltage = pd.read_csv('new전압평균')

일단 평균을 다 뺌 진짜 평균이기 때문에 이미 다른 값들이 다 있기 때문에 평균을 뺌

In [ ]:
powerfactor.drop(['무효전력평균','상전압평균','선간전압평균','역률평균','유효전력평균','전류고조파평균','전류평균','전압고조파평균'],axis = 1,inplace = True)
electric.drop(['무효전력평균','상전압평균','선간전압평균','역률평균','유효전력평균','전류고조파평균','전류평균','전압고조파평균'],axis = 1,inplace = True)
voltage.drop(['무효전력평균','상전압평균','선간전압평균','역률평균','유효전력평균','전류고조파평균','전류평균','전압고조파평균'],axis = 1,inplace = True)

In [ ]:
powerfactor.rename(columns = {'LABEL_NAME':'POWER_LABEL'},inplace = True)
electric.rename(columns = {'LABEL_NAME':'ELECTRIC_LABEL'},inplace = True)
voltage.rename(columns = {'LABEL_NAME':'VOLTAGE_LABEL'},inplace = True)

In [ ]:
data = pd.concat([powerfactor,electric[['ELECTRIC_LABEL']],voltage[['VOLTAGE_LABEL']]],axis = 1)

In [ ]:
# data.to_csv('9월16일LABEL합친데이터')

In [ ]:
data['FINAL_LABEL'] = 0
data

일단 이 데이터는 레이블을 다 합친 데이터


In [ ]:
# 다 같은거 뽑고
same_label = data[(data['POWER_LABEL'] == data['ELECTRIC_LABEL']) &(data['POWER_LABEL'] == data['VOLTAGE_LABEL'])]
same_label['FINAL_LABEL'] = same_label['POWER_LABEL']
same_label.drop(['POWER_LABEL','ELECTRIC_LABEL','VOLTAGE_LABEL'],axis = 1,inplace = True)

In [ ]:
same_label_index = list(same_label.index)
same_label_index
same_label = same_label.reset_index(drop= True)

In [ ]:
# 1 : 1 : 1 뽑는다
all_diff_label = data[(data['POWER_LABEL'] != data['ELECTRIC_LABEL']) &(data['POWER_LABEL'] != data['VOLTAGE_LABEL']) &(data['ELECTRIC_LABEL'] != data['VOLTAGE_LABEL'])]
all_diff_label_index = list(all_diff_label.index)
all_diff_label_index
all_diff_label = all_diff_label.reset_index(drop= True)

In [ ]:
# 2 : 1 뽑는다
# 1 : 1 : 1, 다 같은 데이터를 빼줘서 2대 1을 얻음

total_index = same_label_index + all_diff_label_index

two_one_label = data.drop(total_index,axis = 0)

two_one_label= two_one_label.reset_index(drop=True)

In [ ]:
two_one_label[['POWER_LABEL','ELECTRIC_LABEL','VOLTAGE_LABEL']]

In [ ]:
two_dict = {}
idx = 0
for one,two,three in zip(two_one_label['POWER_LABEL'],two_one_label['ELECTRIC_LABEL'],two_one_label['VOLTAGE_LABEL']):
    two_dict = {'경고' : 0, '정상' : 0, '주의' : 0}
    two_dict[one] += 1
    two_dict[two] += 1
    two_dict[three] += 1
    if two_dict['경고'] == 2:
        two_one_label.loc[idx,'FINAL_LABEL'] = '경고'
    elif two_dict['정상'] == 2:
        two_one_label.loc[idx,'FINAL_LABEL'] = '정상'
    elif two_dict['주의'] == 2:
        two_one_label.loc[idx,'FINAL_LABEL'] = '주의'
    idx += 1
two_one_label

# 2대1인 데이터에서 2를 뽑는 for 문

In [ ]:
two_one_label[['POWER_LABEL','ELECTRIC_LABEL','VOLTAGE_LABEL','FINAL_LABEL']]

elec_count = two_one_label[two_one_label['ELECTRIC_LABEL'] == two_one_label['FINAL_LABEL']].shape[0]
volt_count = two_one_label[two_one_label['VOLTAGE_LABEL'] == two_one_label['FINAL_LABEL']].shape[0]
power_count= two_one_label[two_one_label['POWER_LABEL'] == two_one_label['FINAL_LABEL']].shape[0]

In [ ]:
print(elec_count,volt_count,power_count)

In [ ]:
two_one_label.drop(['POWER_LABEL','ELECTRIC_LABEL','VOLTAGE_LABEL'],axis = 1,inplace = True)
two_one_label

In [ ]:
model_data = pd.concat([same_label,two_one_label])
model_data

In [ ]:
model_data = model_data.sort_values(by = 'TIMESTAMP')

In [ ]:
model_data.to_csv('9월16일model만들때이거써라@@',index = False)

In [ ]:
model_data = pd.read_csv('9월16일model만들때이거써라@@')

In [ ]:
all_diff_label = all_diff_label.sort_values(by = 'TIMESTAMP')

In [ ]:
all_diff_label

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
model_data['FINAL_LABEL'] = encoder.fit_transform(model_data['FINAL_LABEL'])


In [ ]:
encoder.classes_

0이 경고, 1이 정상, 2가 주의

In [ ]:
y_data = model_data['FINAL_LABEL']
x_data = model_data.drop(['FINAL_LABEL','TIMESTAMP'],axis = 1)

In [ ]:
y_data

In [ ]:
from sklearn.metrics import accuracy_score

x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,shuffle = False,random_state = 0)
model = RandomForestClassifier(n_estimators=500)
model.fit(x_train,y_train)
prediction = model.predict(x_test)

print('정확도 : ',accuracy_score(prediction,y_test))

In [ ]:
final_x_data=all_diff_label.drop(['TIMESTAMP','POWER_LABEL','ELECTRIC_LABEL','VOLTAGE_LABEL','FINAL_LABEL'],axis = 1)

In [ ]:
model2 = RandomForestClassifier(n_estimators=500)
model2.fit(x_data,y_data)
prediction2 = model2.predict(final_x_data)


In [ ]:
prediction2
# 2가 뭐고 1

In [ ]:
final_x_data

In [ ]:
all_diff_label['FINAL_LABEL'] = prediction2

In [ ]:
all_diff_label

In [ ]:
label_dict = {0:'경고',1:'정상',2:'주의'}
all_diff_label['FINAL_LABEL'] = all_diff_label['FINAL_LABEL'].map(label_dict)

0이 경고, 1이 정상, 2가 주의

In [ ]:
all_diff_label

In [ ]:
all_diff_label.to_csv('다다른레이블한개로예측한파일',index=False)

In [ ]:
all_diff_label

In [ ]:
elec_count += all_diff_label[all_diff_label['ELECTRIC_LABEL'] == all_diff_label['FINAL_LABEL']].shape[0]
volt_count += all_diff_label[all_diff_label['VOLTAGE_LABEL'] == all_diff_label['FINAL_LABEL']].shape[0]
power_count+= all_diff_label[all_diff_label['POWER_LABEL'] == all_diff_label['FINAL_LABEL']].shape[0]

print(elec_count,volt_count,power_count)

electric이 22169개
voltage 21146개
powerfactor가 986개


In [ ]:
sum = elec_count + volt_count + power_count

print(f'전류고조파 비율 : {elec_count / sum * 100 :.2f} 전압고조파 비율 : {volt_count / sum * 100:.2f} 역률 비율 : {power_count / sum * 100:.2f}')

In [ ]:
elec_count / sum